In [1]:
import json
import pandas as pd
import numpy as np
from ripe.atlas.sagan import PingResult, TracerouteResult

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import ping data
data_path = "measurement_data/traceroute_1.json"

with open(data_path) as f:
    data = json.load(f)
    results = [PingResult(result) for result in data]
    del data
    results_dict = []
    for result in results:
        # create dictionary from result object
        result_dict = result.__dict__
        # remove packets key and make keys for each packet in packets
        # packets = result_dict.pop('packets')
        # for i in range(len(packets)):
        #     packet_dict: dict = packets[i].__dict__
        #     for key, value in packet_dict.items():
        #         result_dict[f'packet_{i+1} {key}'] = value
        results_dict.append(result_dict)
    df_ping: pd.DataFrame = pd.DataFrame(results_dict)

In [ ]:
# Drop columns for ping measure
df_ping["dropped_packets"] = df["packets_sent"] - df["packets_received"]
df_ping = df_ping.drop(["_on_error", "is_error", "seconds_since_sync", "_on_malformation", "is_malformed", 
              "raw_data","error_message", "measurement_id", "firmware", "group_id", 
              "af", "bundle", "step", "packets", "protocol", "origin", 
              "destination_name", "destination_address", "packets_sent", 
              "packets_received", "packet_size"], axis=1)
df = df_ping.dropna()
df_ping

In [ ]:
df_ping = df.reset_index()
df_ping.to_feather("measurement_data/traceroute_1.feather")

In [2]:
# Import Traceroute data
data_path = "measurement_data/traceroute_1.json"

with open(data_path) as f:
    data = json.load(f)
    results = [TracerouteResult(result) for result in data]
    del data
    results_dict = []
    for result in results:
        # create dictionary from result object
        result_dict = result.__dict__
        # remove packets key and make keys for each packet in packets
        # packets = result_dict.pop('packets')
        # for i in range(len(packets)):
        #     packet_dict: dict = packets[i].__dict__
        #     for key, value in packet_dict.items():
        #         result_dict[f'packet_{i+1} {key}'] = value
        results_dict.append(result_dict)
    df_traceroute: pd.DataFrame = pd.DataFrame(results_dict)

In [3]:
# Drop columns for traceroute measure
# df["dropped_packets"] = df["packets_sent"] - df["packets_received"]
df_traceroute = df_traceroute.drop(["_on_error", "is_error", "seconds_since_sync", "_on_malformation", "is_malformed", 
              "raw_data","error_message", "measurement_id", "firmware", "group_id", 
              "af", "bundle", "protocol", "origin", 
              "destination_name", "destination_address", "hops",
              "size"], axis=1)
# df = df.dropna()
df_traceroute

,created,probe_id,source_address,end_time,paris_id,total_hops,last_median_rtt,destination_ip_responded,last_hop_responded,is_success,last_hop_errors
0,2021-11-25 16:14:49+00:00,6025,213.225.160.239,2021-11-25 16:14:55+00:00,15,9,155.201,True,True,True,[]
1,2021-11-25 16:14:59+00:00,6030,193.105.163.172,2021-11-25 16:15:06+00:00,15,12,161.019,True,True,True,[]
2,2021-11-25 16:14:59+00:00,6031,145.220.0.55,2021-11-25 16:15:25+00:00,15,9,152.612,True,True,True,[]
3,2021-11-25 16:14:53+00:00,6041,213.190.96.41,2021-11-25 16:15:26+00:00,15,7,192.820,True,True,True,[]
4,2021-11-25 16:14:58+00:00,6043,192.65.184.54,2021-11-25 16:15:13+00:00,15,9,194.379,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...
345843,2021-11-30 15:00:02+00:00,7030,45.11.104.146,2021-11-30 15:00:17+00:00,10,12,152.791,True,True,True,[]
345844,2021-11-30 15:00:01+00:00,7033,85.93.215.82,2021-11-30 15:00:04+00:00,10,11,145.836,True,True,True,[]
345845,2021-11-30 15:00:04+00:00,7039,103.170.232.152,2021-11-30 15:00:30+00:00,10,9,108.516,True,True,True,[]
345846,2021-11-30 15:00:00+00:00,7045,5.188.202.25,2021-11-30 15:00:00+00:00,10,1,0.091,True,True,True,[]


In [17]:
df_traceroute = df_traceroute.reset_index()
df_traceroute.to_feather("measurement_data/traceroute_1.feather")